In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
data = pd.read_pickle('../data/train.pkl')
data.head()

,$schema,id,type,namespace,title,comment,timestamp,user,bot,minor,...,log_params.details,log_params.oldgroups,log_params.newgroups,log_params.oldmetadata,log_params.newmetadata,log_params.type,log_params.ids,log_params.ofield,log_params.nfield,log_params.restrictions.pages
0,/mediawiki/recentchange/1.0.0,1.576099e+09,edit,0,List of educational institutions in Multan,/* Universities */,1670092463,116.240.127.97,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/mediawiki/recentchange/1.0.0,1.576099e+09,edit,1,Talk:List of awards and honors received by Ron...,wikiprojects,1670092463,Wow,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/mediawiki/recentchange/1.0.0,1.576099e+09,edit,0,Cournot competition,/* Example 1. */,1670092464,Rowing007,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/mediawiki/recentchange/1.0.0,1.576099e+09,categorize,14,Category:Low-importance List articles,[[:Talk:List of awards and honors received by ...,1670092463,Wow,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/mediawiki/recentchange/1.0.0,1.576099e+09,categorize,14,Category:Unassessed United States articles of ...,[[:Talk:List of awards and honors received by ...,1670092463,Wow,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Possible features for bot classification:
- 'bot' in username
- type of change (one-hot encoded)
- whether change is minor
- time between edits
- length of commentary
- content of commentary (tf-idf or Bow)
- length of revision ('length.new'-'length.old')
- Number of requests per minute

In [4]:
data['bot'] = data['bot'].astype(int)

In [8]:
data['bot_in_username'] = data['user'].str.contains('bot', case=False).astype(int)

In [ ]:
data['minor'] = data['minor'].astype(int)

In [ ]:
data['comment_len'] = data['comment'].str.len()

In [ ]:
data['revision_len'] = data['length.new'] - data['length.old']

In [6]:
#one-hot encode type
data = pd.concat([data, data['type'].str.get_dummies()], axis=1)

In [9]:
data['datetime'] = pd.to_datetime(data['timestamp'], unit='s')

In [16]:
# avg changes by user per minute
changes_per_min = data.groupby(['user', pd.Grouper(key='datetime', freq='min')]).size().reset_index()\
    .rename(columns={0:'changes_per_min', 'datetime': 'minute'})
avg_changes_per_min = changes_per_min.groupby('user').agg(avg_changes_per_min=('changes_per_min', 'mean'))
data['minute'] = data['datetime'].dt.round(freq='min')
data = pd.merge(data, changes_per_min, how='left', on=['user', 'minute'])

## Train/test split

In [ ]:
all_users = data['user'].unique().tolist()
train_size = int(len(all_users) * 0.7)
train_users = np.random.choice(all_users, train_size)
train = data[data['user'].isin(train_users)]
test = data[~data['user'].isin(train_users)]

## TFIDF

In [ ]:
tfidf = TfIdfVeectorizer()
tfidf.fit(train['comment'])
train['comment_tfidf'] = tfidf.transform(train['comment'])
test['comment_tfidf'] = tfidf.transform(test['comment'])

In [ ]:
indep_vars = ['bot_in_username', 'minor', 'comment_len', 'revision_len', 'changes_per_min', 'comment_tfidf']

In [ ]:
X_train = train[indep_vars]
Y_train = train['bot']
X_test = test[indep_vars]
Y_test = test['bot']